In [ ]:
#this collaboratory notebook will be used for the code for our project

In [2]:
#*
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#*
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#downloading kaggle file to be able to extract directly from the website -- you need to download a kaggle.json file
#to do so:
#Go to your Kaggle account settings page and click on "Create New API Token". This will download a file called kaggle.json to your local machine.
#upload this file to your directory
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
#*
#reading in CSVs
#dfnofind is the no findings sheet from NIH images
#dfother is allbutnofindings sheet from NIH images
dfnofind = pd.read_csv("./csv_mappings/nofindv2.csv")
dfother = pd.read_csv("./csv_mappings/other_multilabel.csv")

In [4]:
#*
dfother

,image,class
0,00000001_000.png,Cardiomegaly
1,00000001_001.png,Cardiomegaly|Emphysema
2,00000001_002.png,Cardiomegaly|Effusion
3,00000003_000.png,Hernia
4,00000003_001.png,Hernia
...,...,...
51754,00030786_007.png,Consolidation|Pleural_Thickening
51755,00030789_000.png,Infiltration
51756,00030793_000.png,Mass|Nodule
51757,00030795_000.png,Pleural_Thickening


In [5]:
#*
#getting 10,000 samples of no finding using random state = 0
dfsample = dfnofind.sample(n=10000,axis = 0, random_state = 0)

In [6]:
#*
#concatanate the 10,000 no finding samples with the other samples
images = pd.concat([dfother,dfsample])
# summary of images 
images.describe()

,image,class
count,61759,61759
unique,61759,836
top,00000001_000.png,No Finding
freq,1,10000


In [7]:
#*
#resetting the index since concatenating messes it up and just looking at the image names for extraction
imagenames = images['image']
imagenames = imagenames.reset_index(drop = True)
imagenames.describe()

count                61759
unique               61759
top       00000001_000.png
freq                     1
Name: image, dtype: object

In [7]:
#*
#downloading kaggle file for NIH data
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d nih-chest-xrays/data -p /content

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
#this is just showing what the file names are for the next piece of code
path =  './data/nih_data.zip'
import zipfile
with zipfile.ZipFile(path, 'r') as zip_ref:
    # Get a list of all the file names in the zip file
    file_names = zip_ref.namelist()
    print(file_names[0:10])

['ARXIV_V5_CHESTXRAY.pdf', 'BBox_List_2017.csv', 'Data_Entry_2017.csv', 'FAQ_CHESTXRAY.pdf', 'LOG_CHESTXRAY.pdf', 'README_CHESTXRAY.pdf', 'images_001/images/00000001_000.png', 'images_001/images/00000001_001.png', 'images_001/images/00000001_002.png', 'images_001/images/00000002_000.png']


In [24]:
#*
from pandas.core.arrays import string_
import cv2
import shutil

#folder names
paths = ['images_001/images/','images_002/images/','images_003/images/','images_004/images/','images_005/images/','images_006/images/','images_007/images/','images_008/images/','images_009/images/','images_010/images/','images_011/images/','images_012/images/']

# Set path to zip file
zip_path = './data/nih_data.zip'

# Extracted images folder 
extracted_images_path = './data/nih_data/'

# Set path to extract or move the renamed images to
extractpath = './data/dataimages/'

use_zip = False

if use_zip:
  # Open the zip file
  with zipfile.ZipFile(zip_path, 'r') as zip_file:

    #loop through images and paths
      for i in range(0,len(images)):
        for j in range(0,len(paths)):

          #create path name using the path and the image name from the dataframe
          string = paths[j]+imagenames[i]

          # Check if the file exists in the zip file
          if string in zip_file.namelist():

            # Extract the file
            img= zip_file.read(string)

            extr = extractpath+imagenames[i]
            with open(extr, 'wb') as f:
              f.write(img)
            # zip_file.extract(string, path=extractpath)
            break
else:
  # instead of using zip file, use the folder
  for i in range(0,len(images)):
    for j in range(0,len(paths)):

      string = extracted_images_path + paths[j]+imagenames[i]
      if os.path.exists(string):
        shutil.copy(string, extractpath)
        break

In [25]:
#*
#sorting the images in the dataframe to be in order based on the image name (this is how they were saved in the file)
images2 = images.sort_values(by=['image'])
images2= images2.reset_index(drop = True)
images2.describe()

,image,class
count,40963,40963
unique,40963,15
top,00000001_000.png,no finding
freq,1,10000


In [26]:
import os
directory = "./data/images"
classes =  ['no finding', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

if not os.path.exists(directory):
  os.makedirs(directory)

for i in range(0,len(classes)):
  fullpath = directory+'/'+classes[i]
  if not os.path.exists(fullpath):
    os.makedirs(fullpath)


In [27]:
#*
#for this i made folders named from 0-14 to be put into
import os
import shutil

#creating class folders
directory = "./data/images"
classes =  classes = ['no finding', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

#initial folder
if not os.path.exists(directory):
  os.makedirs(directory)

#class folders
for i in range(0,len(classes)):
  fullpath = directory+'/'+classes[i]
  
  if not os.path.exists(fullpath):
    os.makedirs(fullpath)

#sorting images into folders
i = -1
path = "./data/dataimages"

for filename in sorted(os.listdir(path)):
  #don't want to sort this file
  if not filename.endswith(".ipynb_checkpoints"):

    source = path+'/'+filename
    if images2.iloc[i][1] == 'no finding':
      dest_path = './data/images/no finding'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Atelectasis':
      dest_path = './data/images/Atelectasis'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Cardiomegaly':
      dest_path = './data/images/Cardiomegaly'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Consolidation':
      dest_path = './data/images/Consolidation'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Edema':
      dest_path = './data/images/Edema'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Effusion':
      dest_path = './data/images/Effusion'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Emphysema':
      dest_path = './data/images/Emphysema'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Fibrosis':
      dest_path = './data/images/Fibrosis'
      shutil.move(source, dest_path)
      
    elif images2.iloc[i][1] == 'Hernia':
      dest_path = './data/images/Hernia'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Infiltration':
      dest_path = './data/images/Infiltration'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Mass':
      dest_path = './data/images/Mass'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Nodule':
      dest_path = './data/images/Nodule'
      shutil.move(source, dest_path)
    
    elif images2.iloc[i][1] == 'Pleural_Thickening':
      dest_path = './data/images/Pleural_Thickening'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Pneumonia':
      dest_path = './data/images/Pneumonia'
      shutil.move(source, dest_path)

    elif images2.iloc[i][1] == 'Pneumothorax':
      dest_path = './data/images/Pneumothorax'
      shutil.move(source, dest_path)
  i+=1

In [ ]:
#*
#getting kaggle dataset for pneumothorax
!kaggle datasets download -d vbookshelf/pneumothorax-chest-xray-images-and-masks -p /content

100% 4.49G/4.50G [00:31<00:00, 229MB/s]
100% 4.50G/4.50G [00:31<00:00, 153MB/s]


In [ ]:
#just to check file names
with zipfile.ZipFile('./data/pneuthorax.zip', 'r') as zip_ref:
    print(zip_ref.namelist())

In [42]:
#*
#importing csv from the pneumothorax sheet from NIH jamges which has the names of images that have a pneumothorax
#NOTE this is the pictures not the masks
thorax = pd.read_csv("./csv_mappings/pneumothorax.csv")
thorax.describe()

,image
count,2669
unique,2669
top,2_train_1_.png
freq,1


In [46]:
#*
zip_path = './data/pneuthorax.zip'

# Set path to extract the zip file to
extractpath = './data/images/Pneumothorax'

#put all images from the csv into the pneumothorax file
path = './data/siim-acr-pneumothorax/png_images/'

use_zip = False
if use_zip:
  # Open the zip file
  with zipfile.ZipFile(zip_path, 'r') as zip_file:
      for i in range(0,len(thorax)):
          string = path+thorax['image'].iloc[i]

          img= zip_file.read(string)

          extr = extractpath+thorax['image'].iloc[i]
          with open(extr, 'wb') as f:
            f.write(img)
else:
  # otherwise copy the respective file from `path`
  for i in range(0,len(thorax)):
    string = path+thorax['image'].iloc[i]
    shutil.copy(string, extractpath)

In [ ]:
#*
#getting pneumonia dataset from kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p /content

100% 2.29G/2.29G [00:14<00:00, 139MB/s]
100% 2.29G/2.29G [00:14<00:00, 175MB/s]


In [40]:
#*
#extract it to see the different folder names
import zipfile
with zipfile.ZipFile('./data/chest_xray.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/')

In [ ]:
#*
import shutil

#defining paths and folder names
folders = ['train/PNEUMONIA', 'val/PNEUMONIA','test/PNEUMONIA']
src_dir = './data/chest_xray/'
dst_dir = './data/images/Pneumonia'

#move all images from the train, validation and testing folders to the pneumonia folder
for i in range(0,len(folders)):
  dir = src_dir+folders[i]
  # loop through all files in the source directory
  for filename in os.listdir(dir):
          # create the full path to the source and destination files
          src_file = os.path.join(dir, filename)
          dst_file = os.path.join(dst_dir, filename)
          print(src_file, dst_file)
          # move the file from the source to the destination folder
          shutil.move(src_file, dst_file)

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim #for gradient descent
import torchvision.transforms as transforms
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [2]:
#*
#removes the .ipnb file which will mess up pytorch if not removed
!find . -name ".ipynb_checkpoints" -exec rm -r {} +

Access denied - .
File not found - -NAME
File not found - -EXEC
File not found - RM
File not found - -R
File not found - {}
File not found - +


In [ ]:
#TRANSFER LEARNING
#get dataset from drive
#download checkpoint from the github repo
#load densenet121
#input the checkpoint weights into the densenet 
#use densenet.features() for each tensor
#save to files like lab 3 -- every tensor is saved in a folder for their respective class

In [2]:
p = './data/images'

#tensor transformation
datatransform = transform=transforms.Compose([
                                        transforms.Lambda(lambda x: x.convert('RGB')),
                                        transforms.Resize(224),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor()])

dataset = torchvision.datasets.ImageFolder(p, transform=datatransform)

In [4]:
#*
#MUST SET SEED TO 26
torch.manual_seed(26)
tr,val,te = torch.utils.data.random_split(dataset, [28743,9581,9581],)

In [5]:
#normalizing validataion and testing, but not normalizing training yet since it will be rotated etc first
datatransform = transform=transforms.Compose([
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
val.transform = transform
te.transform = transform

In [7]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from torch.utils.data import Dataset

class OversampledImageFolder(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.targets = [label for _, label in dataset]
        self.oversampler = RandomOverSampler(sampling_strategy='not majority')
        indices = np.arange(len(self.targets)).reshape(-1, 1)
        self.indices, self.targets = self.oversampler.fit_resample(indices, self.targets)
    
    def __getitem__(self, index):
        idx = int(self.indices[index])
        return self.dataset[idx]
    
    def __len__(self):
        return len(self.indices)

In [10]:
tr_oversampled = OversampledImageFolder(tr)
train = tr_oversampled

In [ ]:
print(f'Original class distribution: {Counter(tr.dataset.targets)}, {len(tr.dataset.targets)}')
print(f'Oversampled class distribution: {Counter(tr_oversampled.targets)}, {len(tr_oversampled.targets)}')

Original class distribution: Counter({14: 10000, 8: 9547, 13: 4863, 12: 4595, 0: 4215, 4: 3955, 10: 2705, 9: 2139, 2: 1310, 11: 1126, 1: 1093, 5: 892, 6: 727, 3: 628, 7: 110}), 47905
Oversampled class distribution: Counter({4: 6000, 14: 6000, 9: 6000, 12: 6000, 8: 6000, 13: 6000, 0: 6000, 11: 6000, 6: 6000, 10: 6000, 2: 6000, 5: 6000, 1: 6000, 7: 6000, 3: 6000}), 90000


In [11]:
#transformation to do a rotation and flip and also normalize the training dataset
#EVEN IF YOU DON"T WANT TO ROTATE OR FLIP MAKE SURE TO AT LEAST RUN THE NORMALIZE
transform = transforms.Compose([
    transforms.RandomRotation((-15,15)),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train.transform = transform

In [12]:
trloader = torch.utils.data.DataLoader(train, batch_size=1, shuffle = True)
valloader = torch.utils.data.DataLoader(val, batch_size=1,shuffle = True)
testloader = torch.utils.data.DataLoader(te, batch_size=1, shuffle = True)

In [13]:
#make sure to load the model.pth.tar checkpoint to use
import torchvision.models as models
checkpoint = torch.load('./model.pth.tar',map_location=torch.device('cpu'))

#loading the dictionary of the checkpoint and loading the densent model
model = models.densenet121(pretrained=True)
model_dict = model.state_dict()
saved_state_dict = checkpoint['state_dict']

# Modify the keys in the saved state dict to match the keys in your model
newdict = {}
for key, value in saved_state_dict.items():
    new_key = key.replace('densenet121.', '')
    new_key = new_key.replace('norm.', 'norm')
    new_key = new_key.replace('conv.', 'conv')
    new_key = new_key.replace('normr', 'norm.r')
    new_key = new_key.replace('normb', 'norm.b')
    new_key = new_key.replace('normw', 'norm.w')
    new_key = new_key.replace('convw', 'conv.w')
    newdict[new_key] = value

#ignoring the model checkpoint's classifiers
model_dict = model.state_dict()
checkpoint_dict = {k: v for k, v in newdict.items() if k in model_dict}
model_dict.update(checkpoint_dict)

#loading in the model dictionary
model.load_state_dict(model_dict)

c:\ProgramData\Anaconda3\envs\pytorch-torchtext\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\pytorch-torchtext\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [14]:
#* sanity check should get [1,3,224,224] and [1,1024,7,7]
for imgs, labels in iter(valloader):
    print(imgs.size())
    f = model.features(imgs)
    print(f.size())
    break

torch.Size([1, 3, 224, 224])
torch.Size([1, 1024, 7, 7])


In [15]:
#creating directories
folders = ['embeddingtrain','embeddingval', 'embeddingtest']
directory = "./embeddings/"
classes = list(dataset.class_to_idx.keys())

#you need a separate folder for train, val and test with all 15 folders in each
for i in range(0,len(folders)):
  d = directory+folders[i]
  #initial folder
  if not os.path.exists(d):
    os.makedirs(d)

  #class folders
  for i in range(0,len(classes)):
    fullpath = d+'/'+classes[i]
  
    if not os.path.exists(fullpath):
      os.makedirs(fullpath)

In [16]:
#ensure this is run 
classes = list(dataset.class_to_idx.keys())
folders = ['embeddingtrain','embeddingval', 'embeddingtest']

In [17]:
import os
import shutil

#creating class folders

def get_features(loader,classes,folder):
  path ='./embeddings/'+folder
  n = 0
  for imgs, labels in iter(loader):

    label = classes[int(labels[0])]
    features = model.features(imgs)
    features_tensor = torch.from_numpy(features.detach().numpy())
    torch.save(features_tensor.squeeze(0), path +  '/' +label + '/' + str(n)+ '.tensor')
    n+=1
    if n%1000==0:
      print(n)

In [18]:
get_features(trloader,classes,folders[0])
get_features(valloader,classes,folders[1])
get_features(testloader,classes,folders[2])

: 

: 